<a href="https://www.comet.com/site/?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth" >
  <img src="https://cdn.comet.ml/img/notebook_logo.png">
</a>

# Comet and unsloth

[Comet](https://www.comet.com/site/?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth) is an MLOps platform designed to help data scientists and teams build better models faster! Comet provides tooling to track, explain, manage, and monitor your models in a single place! It works with Jupyter notebooks and scripts and-- most importantly--it's 100% free to get started!

[unsloth](https://github.com/unslothai/unsloth) dramatically improves the speed and efficiency of LLM fine-tuning for models including Llama, Phi-3, Gemma, Mistral, and more. For a full listed of 100+ supported unsloth models, [see here](https://huggingface.co/unsloth).

Instrument your torchtune training runs with Comet to start managing experiments with efficiency, reproducibility, and collaboration in mind.

Find more information about [our integration with torchtune here](https://www.comet.com/docs/v2/integrations/third-party-tools/unsloth?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth) or [learn about our other integrations here](https://www.comet.com/docs/v2/integrations?utm_medium=colab&utm_source=comet-examples&utm_campaign=unsloth).

## ⚙ Install and import dependencies

In [1]:
%%capture
!pip install comet_ml "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# check torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [5]:
import comet_ml

comet_ml.login()
exp = comet_ml.Experiment(project_name="comet-example-unsloth", workspace="examples")

COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/examples/comet-example-unsloth/cb280e1a2ac942cbab2343349d80282f



In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## ⚙ Download model

In [7]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Find the full list of [100+ supported unsloth models here](https://huggingface.co/unsloth). For a full list of supported 4-bit models see [here](https://huggingface.co/collections/unsloth/load-4bit-models-4x-faster-659042e3a41c3cbad582e734)

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = HF_TOKEN
)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## ⚙ Add LoRA adapters

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## ⚙ Data preparation

In [10]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # add EOS_TOKEN, otherwise your generation will go on forever
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

## ⚙ Training

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.818600
2,2.305800
3,1.703400
4,2.014700
5,1.735100
6,1.670600
7,1.259200
8,1.308900
9,1.165100
10,1.225300


In [13]:
exp.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : apparent_pagoda_1033
COMET INFO:     url                   : https://www.comet.com/examples/comet-example-unsloth/cb280e1a2ac942cbab2343349d80282f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     epoch [61]                     : (0.0001545595054095827, 0.00927357032457496)
COMET INFO:     grad_norm [60]                 : (0.2999851107597351, 2.063112258911133)
COMET INFO:     learning_rate [60]             : (0.0, 0.0002)
COMET INFO:     loss [60]                      : (0.7567, 2.3058)
COMET INFO:     total_flos                     : 5726714157219840.0
COMET INFO:     train/epoch [61]               : (0.0001545595054095827, 0.00927357032457

## ⚙ Inference

In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']